# Race for Autonomy - EMA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ema_workbench
import datetime
import random
import math

from ema_workbench import load_results
from ema_workbench.analysis import pairs_plotting
import matplotlib.pyplot as plt
from ema_workbench.analysis import prim
import ema_workbench.analysis.clusterer as clusterer
from sklearn.cluster import AgglomerativeClustering

%matplotlib inline
from ema_workbench import(Model, RealParameter,Constant,IntegerParameter,ScalarOutcome,CategoricalParameter, TimeSeriesOutcome, Policy, perform_experiments, ema_logging, save_results, load_results)
from ema_workbench.connectors.vensim import VensimModel
from ema_workbench.analysis.plotting import lines, plot_lines_with_envelopes,envelopes
from ema_workbench.analysis import Density
import ema_workbench.analysis.pairs_plotting as pairs
import ema_workbench.analysis.plotting as emaplt

In [ ]:
#get model
wd =  './model/'
model = VensimModel("simpleModel", wd=wd, model_file="AutonomyRace.vpmx")

# Prepare EMA

Define uncertainties depending on analysis (extreme conditions validation or analysis).

In [ ]:
# prepare uncertainties are categoricals for exgtreme conditions (minimum and maximum values)

uncertainties = [
CategoricalParameter('Initial UAS stock A',[500,10000]), 
CategoricalParameter('Initial tech sophistication under development A',[0.125,0.6]),
CategoricalParameter('Initial UAS stock B',[50,3000]),
CategoricalParameter('Initial tech sophistication under development B', [0.125,0.5]),
CategoricalParameter('Initial level of autonomy under development A',[0.125,0.6]),
CategoricalParameter('Initial level of autonomy A',[0.05,0.4]),
CategoricalParameter('Initial level of tech sophistication B',[0.005,0.2]),
CategoricalParameter('Initial level of autonomy under development B',[0.1,0.5]),
CategoricalParameter('Initial level of autonomy B',[0.005,0.2]),
CategoricalParameter('Initial level of tech sophistication A',[0.05,0.4]),
CategoricalParameter('v tech A',[1,20]), 
CategoricalParameter('v tech B',[1,20]),
CategoricalParameter('Q tech A',[1,20]), 
CategoricalParameter('Q tech B',[1,20]), 
CategoricalParameter('v AI A',[1,20]), #lever
CategoricalParameter('v AI B',[1,20]),#lever
CategoricalParameter('Q AI A',[1,20]), #lever
CategoricalParameter('Q AI B',[1,20]),#lever
CategoricalParameter('Initial capabilities A',[50000,400000]),
CategoricalParameter('Initial capabilities B',[5000,300000]), 
CategoricalParameter('Sensitivity to opposing UAS production A',[0.375,2.5]), 
CategoricalParameter('Sensitivity to opposing UAS production B',[0.375,2.5]),
CategoricalParameter('Average life of UAS A',[10,60]),
CategoricalParameter('Average lifetime of capabilities A',[10,60]),
CategoricalParameter('Average lifetime of capabilities B',[2.5,50]),
CategoricalParameter('Average life of UAS B',[2.5,50]),
CategoricalParameter('Capability procurement speed A',[1,14]),
CategoricalParameter('Average procurement delay other cap A',[1,14]),
CategoricalParameter('Capability procurement speed B',[3.5,20]),
CategoricalParameter('Average procurement delay other cap B',[3.5,20]),
CategoricalParameter('Time to change desired margin of superiority A',[0.375,4]), 
CategoricalParameter('Time to authorize UAS development A',[0.375,4]),
CategoricalParameter('Time to authorize UAS development B',[0.375,4]),
CategoricalParameter('Time to perceive opposing UAS A',[0.125,1.5]),
CategoricalParameter('Time to perceive opposing UAS B',[0.125,1.5]),
CategoricalParameter('Average procurement delay A',[0.5,40]),
CategoricalParameter('Average procurement delay B',[0.5,40]),
CategoricalParameter('GDP growth rate A',[0.005,0.14]), 
CategoricalParameter('GDP growth rate B',[0.0025,0.2]),
CategoricalParameter('Initial GDP A',[1000000000000,40000000000000]),
CategoricalParameter('Initial GDP B',[400000000000,4000000000000]),
CategoricalParameter('Initial Defense budget as part of GDP A',[0.005,0.2]),
CategoricalParameter('Initial Defense budget as part of GDP B',[0.005,0.2]),
CategoricalParameter('Base operators needed A',[0.25,20]), #lever
CategoricalParameter('Base operators needed B',[0.005,4]), #lever
CategoricalParameter('maintainers per UAS A',[0.005,10]),
CategoricalParameter('maintainers per UAS B',[0.005,10]),
CategoricalParameter('Time to recruit A',[0.25,4]),
CategoricalParameter('Time to recruit B',[0.25,4]),
CategoricalParameter('Time to train A',[0.5,10]),
CategoricalParameter('Time to train B',[0.5,4]),
CategoricalParameter('Time in service A',[3.5,50]),
CategoricalParameter('Time in service B',[2.5,50]),
CategoricalParameter('Training cost per personnel A',[2500,50000]),
CategoricalParameter('Training cost per personnel B',[500,20000]),
CategoricalParameter('Cost per personnel A',[25000,500000]),
CategoricalParameter('Cost per personnel B',[5000,200000]),
CategoricalParameter('initial percentage of military budget for autonomous systems A',[0.0005,0.9]), #not working with 100%
CategoricalParameter('initial percentage of military budget for autonomous systems B',[0.0005,0.9]),
CategoricalParameter('Sensitivity to opposing UAS production A',[0.375,2.5]),
CategoricalParameter('Bias in estimating opposing UAS A',[0.375,2.5]),
CategoricalParameter('Sensitivity to opposing UAS production B',[0.375,2.5]),
CategoricalParameter('Bias in estimating opposing UAS B',[0.375,2.5]),
CategoricalParameter('weight autonomy B',[0.5,1]),
CategoricalParameter('weight autonomy A',[0.5,1]),
CategoricalParameter('Bias in estimating domestic UAS A',[0.375,2.5]),
CategoricalParameter('Bias in estimating domestic UAS B',[0.375,2.5])]

In [ ]:
# prepare uncertainties as parameter ranges for analyses


uncertainties = [
RealParameter('Initial UAS stock A',1000,5000), 
RealParameter('Initial tech sophistication under development A', 0.25,0.3),
RealParameter('Initial UAS stock B',100,1500),
    
RealParameter('Initial tech sophistication under development B', 0.2,0.25),
RealParameter('Initial level of autonomy under development A',0.25,0.3),
RealParameter('Initial level of autonomy A',0.1,0.2),
RealParameter('Initial level of tech sophistication B',0.01,0.1),
RealParameter('Initial level of autonomy under development B',0.2,0.25),
RealParameter('Initial level of autonomy B',0.01,0.1),
RealParameter('Initial level of tech sophistication A',0.1,0.2),


RealParameter('v tech A',1,20), 
RealParameter('v tech B',1,20),
RealParameter('Q tech A',1,20), 
RealParameter('Q tech B',1,20), 
RealParameter('v AI A',1,20), #lever
RealParameter('v AI B',1,20),#lever
RealParameter('Q AI A',1,20), #lever
RealParameter('Q AI B',1,20),#lever


RealParameter('Initial capabilities A',100000,200000),
RealParameter('Initial capabilities B',10000,150000), 

RealParameter('Sensitivity to opposing UAS production A',0.75,1.25), 
RealParameter('Sensitivity to opposing UAS production B',0.75,1.25),
    
RealParameter('Average life of UAS A',20,30),
RealParameter('Average lifetime of capabilities A',20,30),
    
RealParameter('Average lifetime of capabilities B',5,25),
RealParameter('Average life of UAS B',5,25),
  
    
RealParameter('Capability procurement speed A',2,7),
RealParameter('Average procurement delay other cap A',2,7),
RealParameter('Capability procurement speed B',7,10),
RealParameter('Average procurement delay other cap B',7,10),
    

RealParameter('Time to change desired margin of superiority A',0.75,2), 
RealParameter('Time to authorize UAS development A',0.75,2),
RealParameter('Time to authorize UAS development B',0.75,2),
RealParameter('Time to perceive opposing UAS A',0.25,0.75),
RealParameter('Time to perceive opposing UAS B',0.25,0.75),

RealParameter('Average procurement delay A',2,10),
RealParameter('Average procurement delay B',2,10),
RealParameter('GDP growth rate A',0.01,0.07), 
RealParameter('GDP growth rate B',0.05,0.1),
RealParameter('Initial GDP A',2000000000000,20000000000000),
RealParameter('Initial GDP B',800000000000,2000000000000),
RealParameter('Initial Defense budget as part of GDP A',0.01,0.1),
RealParameter('Initial Defense budget as part of GDP B',0.01,0.1),

RealParameter('Base operators needed A',0.5,10), #lever
RealParameter('Base operators needed B',0.01,2), #lever
RealParameter('maintainers per UAS A',0.01,5),
RealParameter('maintainers per UAS B',0.01,5),
    
RealParameter('Time to recruit A',0.5,2),
RealParameter('Time to recruit B',0.5,2),
RealParameter('Time to train A',1,5),
RealParameter('Time to train B',1,2),
RealParameter('Time in service A',7,25),
RealParameter('Time in service B',5,25),
RealParameter('Training cost per personnel A',5000,25000),
RealParameter('Training cost per personnel B',1000,10000),
RealParameter('Cost per personnel A',50000,250000),
RealParameter('Cost per personnel B',10000,100000),


RealParameter('initial percentage of military budget for autonomous systems A',0.001,0.99),
RealParameter('initial percentage of military budget for autonomous systems B',0.001,0.99),
RealParameter('Sensitivity to opposing UAS production A',0.75,1.25),
RealParameter('Bias in estimating opposing UAS A',0.75,1.25),
RealParameter('Sensitivity to opposing UAS production B',0.75,1.25),
RealParameter('Bias in estimating opposing UAS B',0.75,1.25),
RealParameter('weight autonomy B',0.5,1),
RealParameter('weight autonomy A',0.5,1),
RealParameter('Bias in estimating domestic UAS A',0.75,1.25),
RealParameter('Bias in estimating domestic UAS B',0.75,1.25)]

In [ ]:
outcomes = [
TimeSeriesOutcome('New percentage of GDP B'),
TimeSeriesOutcome('New percentage of GDP A'),
TimeSeriesOutcome('UAS stock A'),
TimeSeriesOutcome('UAS stock B'),
TimeSeriesOutcome('effectiveness UAS stock A'),
TimeSeriesOutcome('effectiveness UAS stock B'),
]

In [ ]:
model.uncertainties = uncertainties
model.outcomes = outcomes

# Define policies

Define policies (either regulation policies or spoiling policies).

To generate the vpmx files for the regulation policies:
- in the Vensim Model file, go to policy switch "Regulations" in the Technology submodel
- restrict the level of autonomy to 5 as indicated in the comment box of this parameter

To generate the vpmx files for the spoiling policies:
- for the quality policy, in Technology submbodel, switch policy lever for "take out skilled personnel" and "restrict access to critical components" to 1
- for the funding policy, in Racing submodel, switch policy lever for "restrict access to funding" to 1.
- for the combined policy, do both

Run the model in these configurations independently and export vpmx files for each model run.

In [ ]:
# regulations

policies = [Policy('no policy',
                   model_file='AutonomyRace.vpmx'),
            Policy('adaptive policy',
                   model_file='AutonomyRaceAdaptive.vpmx'),
            Policy('halfway policy',
                   model_file='AutonomyRaceHalfway.vpmx'),
            Policy('static policy',
                   model_file='AutonomyRaceStatic.vpmx')
            ]

In [ ]:
# spoiling strategies

policies =  [Policy('no policy',
                   model_file='AutonomyRace.vpmx'),
            Policy('quality policy',
                   model_file='AutonomyRaceQuality.vpmx'),
            Policy('funding policy',
                   model_file='AutonomyRaceFunding.vpmx'),
            Policy('combined policy',
                   model_file='AutonomyRaceCombined.vpmx'),
            ]

# Open exploration / Extreme Conditions

In [ ]:
nbscenarios = 1000
experiments, outcomes  = perform_experiments(model,  nbscenarios = nbscenarios, policies=policies)
#ema_workbench.util.utilities.save_results(results, "1000Scenarios_4Policies")
#results = ema_workbench.util.utilities.load_results("Results")
results = experiments, outcomes


In [ ]:
# show the outcomes of interest
plt.rcParams["figure.figsize"] = (50,50)
fig, axes = pairs_plotting.pairs_scatter(experiments, outcomes, group_by='policy', legend=False)
fig.set_size_inches(20, 20)
plt.show()

In [ ]:
# plot the spendings
spendings = {}
spendings['UAS stock A'] = outcomes['UAS stock A']
spendings['UASstock B'] = outcomes['UAS stock B']

fig, axes = pairs_plotting.pairs_scatter(experiments, spendings, group_by='policy', legend=False)
figure = lines(experiments, spendings)#, density = kde) #show lines, and end state density

plt.rcParams["figure.figsize"] =(10,10)

plt.show() #show figure

# Find extreme scenarios

Extreme scenarios are those where a nation spends more than 46% of its GDP as defense.

In [ ]:
#extreme spendings
threshold = 0.46
ExtremeScenarios = []
for column in range(nbscenarios):
    for row in range(7681):
        if (outcomes['New percentage of GDP A'][column][row] > threshold  ) or ( outcomes['New percentage of GDP B'][column][row] > threshold):
            ExtremeScenarios.append(column)
            
ExtremeScenariosAll = list(dict.fromkeys(ExtremeScenarios))
len(ExtremeScenariosAll)/nbscenarios # how many is that?

In [ ]:
#those scenarios with normal defense spendings that are less than 46% of GDP
list1 = np.arange(len(outcomes['New percentage of GDP A'])).tolist()
SetScenariosNotExtreme = set(list1) - set(ExtremeScenarios)

ScenariosNotExtreme = []
for i in SetScenariosNotExtreme:
    ScenariosNotExtreme.append(i)
    
ScenariosNotExtremeArray = np.array(ScenariosNotExtreme)

In [ ]:
## Check out how many extreme cases depending on used policy

In [ ]:
nbExtremeNoP = len(ExtremeScenariosAllArray[ExtremeScenariosAllArray < 1000])  # no policy
nbExtremeNoP/nbscenarios

In [ ]:
nbExtremeCombined = len(ExtremeScenariosAllArray[ExtremeScenariosAllArray > 3000])  # combined policy
nbExtremeCombined/nbscenarios

In [ ]:
# plot no policy
tmp = ScenariosNotExtremeArray[ScenariosNotExtremeArray < 1000 ]

ooiCombined = {}
ooiCombined['New percentage of GDP A'] = outcomes['New percentage of GDP A'][tmp]
ooiCombined['New percentage of GDP B'] = outcomes['New percentage of GDP B'][tmp]

figure = lines(experiments.iloc[tmp], ooiCombined)#, density = kde) #show lines, and end state density
plt.rcParams["figure.figsize"] =(10,10)
#fig.set_size_inches(5, 5)

plt.show() #show figure

In [ ]:
# plot combined policy
tmp = ScenariosNotExtremeArray[ScenariosNotExtremeArray > 3000 ]

ooiCombined = {}
ooiCombined['Stock A'] = outcomes['UAS stock A'][tmp]
ooiCombined['Stock B'] = outcomes['UAS stock B'][tmp]

figure = lines(experiments.iloc[tmp], ooiCombined)#, density = kde) #show lines, and end state density
plt.rcParams["figure.figsize"] =(5,5)
#fig.set_size_inches(5, 5)

plt.show() #show figure

In [ ]:
#plot one nice scenario
df1 = pd.Series( outcomes['New percentage of GDP A'][combinedRaceB[BwinsBRace][0][1]]) 
df2 = pd.Series( outcomes['New percentage of GDP B'][combinedRaceB[BwinsBRace][0][1]])
df1.plot(label='New percentage of GDP A')
df2.plot(label='New percentage of GDP B')
plt.legend()
plt.title("Evolution of the defense spendings of nations A and B")
plt.xlabel("Time Steps")
plt.ylabel("New percentage of GDP")
plt.show()

# Find Arms Races

Arms races are those scenarios where nations increase they defense spending by more than 50% over 5 years.

In [ ]:
GDPa = pd.DataFrame(outcomes['New percentage of GDP A'])
GDPb = pd.DataFrame(outcomes['New percentage of GDP B'])

In [ ]:
#comppute growht races
growthRatesA = []
growthRatesB = []
for i in range(len(GDPa)):
    growthRatesA.append(GDPa.iloc[i,:].pct_change())
    growthRatesB.append(GDPb.iloc[i,:].pct_change())
    
growthRatesDFA = pd.DataFrame(growthRatesA)
growthRatesDFB = pd.DataFrame(growthRatesB)

In [ ]:
# convert growth rates from time steps into years
oneyear = 256 #time steps
growthRatesYearA = []
growthRatesYearB = []
j = 0
for i in range(30):
    growthRatesYearA.append(growthRatesDFA.iloc[:,j:j+oneyear].sum(axis = 1))
    growthRatesYearB.append(growthRatesDFB.iloc[:,j:j+oneyear].sum(axis = 1))
    j += oneyear
growthRatesYearDFA = pd.DataFrame(growthRatesYearA).T
growthRatesYearDFB = pd.DataFrame(growthRatesYearB).T

In [ ]:
# sum up every combination of 5 years that exists within 30 years, which is 26
duration = 5 # or 5, whatever you define an arms race is
ArmsRacesA= []
ArmsRacesB= []
j = 0
while j <= 30 - duration:
    ArmsRacesA.append(growthRatesYearDFA.iloc[:,j:j+duration].sum(axis = 1))
    ArmsRacesB.append(growthRatesYearDFB.iloc[:,j:j+duration].sum(axis = 1))
    j = j + 1
ArmsRacesDFA = pd.DataFrame(ArmsRacesA).T
ArmsRacesDFB = pd.DataFrame(ArmsRacesB).T

In [ ]:
#ALL ARMS RACES
threshold = 0.5
RacingScenarios = []
nb_scenarios = len(ArmsRacesDFB)
for column in range(30 - duration + 1):
    for row in range(nb_scenarios):
        if (ArmsRacesDFA.iloc[row, column] > threshold) or (ArmsRacesDFB.iloc[row, column] > threshold):
            RacingScenarios.append(row)
Races = (np.unique(RacingScenarios))
len(Races)

In [ ]:
#BILATERAL ARMS RACES
threshold = 0.5
RacingScenarios = []
nb_scenarios = len(ArmsRacesDFB)
for column in range(30 - duration + 1):
    for row in range(nb_scenarios):
        if (ArmsRacesDFA.iloc[row, column] > threshold) and (ArmsRacesDFB.iloc[row, column] > threshold):
            RacingScenarios.append(row)
bilateralRaces = (np.unique(RacingScenarios))
len(bilateralRaces)

In [ ]:
nbArmsRacesCombined=len(Races[Races > 3000]) # combined
nbArmsRacesCombined/nbscenarios

In [ ]:
nbArmsRacesNoPolicy=len(Races[Races < 1000]) #no policy
nbArmsRacesNoPolicy/nbscenarios

In [ ]:
len(Races)/nbscenarios

In [ ]:
len(bilateralRaces)/len(Races)

In [ ]:
# Driven by A
threshold = 0.5
RacingScenarios = []
nb_scenarios = len(ArmsRacesDFB)
for column in range(30 - duration + 1):
    for row in range(nb_scenarios):
        if (ArmsRacesDFA.iloc[row, column] > threshold):
            RacingScenarios.append(row)
ARaces = (np.unique(RacingScenarios))

In [ ]:
## those that are only driven by A and are not bilateral ones
onlyARaces = np.setxor1d(ARaces, bilateralRaces)
len(onlyARaces)

In [ ]:
# Driven by B
threshold = 0.5
RacingScenarios = []
nb_scenarios = len(ArmsRacesDFB)
for column in range(30 - duration + 1):
    for row in range(nb_scenarios):
        if (ArmsRacesDFB.iloc[row, column] > threshold):
            RacingScenarios.append(row)
BRaces = (np.unique(RacingScenarios))

In [ ]:
# those that are only driven by B
onlyBRaces = np.setxor1d(BRaces, bilateralRaces)
len(onlyBRaces)

In [ ]:
len(onlyBRaces)/len(Races)

In [ ]:
len(onlyARaces)/len(Races)

In [ ]:
#How many scenarios is that?
len(Races)/len(ArmsRacesDFA)

#  Who wins??

The winner is the nation that by the end of the simulation time has the greatest quantity weighted by their effectiveness.

In [ ]:
# get indices of experiments per policy
funding = np.array(range(2000, 3000))
combined = np.array(range(3000, 4000))

In [ ]:
# get those scenarios where there are no races happening
SetScenariosNotRaces = []
SetScenariosNotRaces = set(np.array(range(nbscenarios)))- set(Races) # - set(ExtremeScenariosAll)
ScenariosNotRaces = []
for i in SetScenariosNotRaces:
    ScenariosNotRaces.append(i)

In [ ]:
# races
# quality policy
qualityRace = Races[Races > 1000 ]
qualityRace = qualityRace[qualityRace < 2000]
# funding policy
fundingRace = Races[Races > 2000 ]
fundingRace = fundingdRace[fundingdRace < 3000]
#combined
combinedRace = Races[Races > 3000 ]


#bilatearl races
#quality policy
qualityRacebilateral = bilateralRaces[bilateralRaces > 1000 ]
qualityRacebilateral = qualityRacebilateral[qualityRacebilateral < 2000]
#funding
fundingdRacebilateral = bilateralRaces[bilateralRaces > 2000 ]
fundingdRacebilateral = fundingdRacebilateral[fundingdRacebilateral < 3000]
#combined
combinedRacebilateral = bilateralRaces[bilateralRaces > 3000 ]


#A driven races
#quality
qualityRaceA = onlyARaces[onlyARaces > 1000 ]
qualityRaceA = qualityRaceA[qualityRaceA < 2000]
#funding
fundingdRaceA = onlyARaces[onlyARaces > 2000 ]
fundingdRaceA = fundingdRaceA[fundingdRaceA < 3000]
#combined
combinedRaceA= onlyARaces[onlyARaces > 3000 ]



#B races
#quality
qualityRaceB = onlyBRaces[onlyBRaces > 1000 ]
qualityRaceB = qualityRaceB[qualityRaceB < 2000]
#funding
fundingRaceB = onlyBRaces[onlyBRaces > 2000 ]
fundingRaceB = fundingdRaceB[fundingdRaceB < 3000]
#combined
combinedRaceB= onlyBRaces[onlyBRaces > 3000 ]

# scenarios where no races happen
#quality
noQualityRace = np.array(ScenariosNotRaces)[np.array(ScenariosNotRaces) > 1000]
noQualityRace = noQualityRace[noQualityRace < 2000]
#funding
nofundingRace = np.array(ScenariosNotRaces)[np.array(ScenariosNotRaces) > 2000]
nofundingRace = nofundingRace[nofundingRace < 3000]
#combined
noCombinedRace = np.array(ScenariosNotRaces)[np.array(ScenariosNotRaces) > 3000]

In [ ]:
# compute ratios of UAS to later check out who wins depending on the policy
UASratioRaces = []
UASratiobilateral = []
UASratioARaces = []
UASratioBRaces = []
UASratioNoRaces = []

for i in combinedRace: # or Race, qualityRace, fundingRace
    UASratioRaces.append((outcomes['UAS stock A'][i][7680]*outcomes['effectiveness UAS stock A'][i][7680])/(outcomes['UAS stock B'][i][7680] * outcomes['effectiveness UAS stock B'][i][7680]))
for i in combinedRacebilateral: # or bilateralRaces, fundingdRacebilateral, qualityRacebilateral
    UASratiobilateral.append((outcomes['UAS stock A'][i][7680]*outcomes['effectiveness UAS stock A'][i][7680])/(outcomes['UAS stock B'][i][7680] * outcomes['effectiveness UAS stock B'][i][7680]))
for i in combinedRaceA: # or onlyARaces, fundingRaceA, qualityRaceA
    UASratioARaces.append((outcomes['UAS stock A'][i][7680]*outcomes['effectiveness UAS stock A'][i][7680])/(outcomes['UAS stock B'][i][7680] * outcomes['effectiveness UAS stock B'][i][7680]))
for i in combinedRaceB: # or onlyBRaces, fundingRaceB, qualityRaceB
    UASratioBRaces.append((outcomes['UAS stock A'][i][7680]*outcomes['effectiveness UAS stock A'][i][7680])/(outcomes['UAS stock B'][i][7680] * outcomes['effectiveness UAS stock B'][i][7680]))
for i in noCombinedRace: # or in np.array(ScenariosNotRaces)[np.array(ScenariosNotRaces) >0], nofundingRace, noQualityRace
    UASratioNoRaces.append((outcomes['UAS stock A'][i][7680]*outcomes['effectiveness UAS stock A'][i][7680])/(outcomes['UAS stock B'][i][7680] * outcomes['effectiveness UAS stock B'][i][7680]))


In [ ]:
# depending of type of race, compute who wins
def racetypeToCheck(ListRace):
    
    UASratioRacesArray = np.array(ListRace)
    UASratioRacesArray# funding

    percentageAwins = np.sum(x > 1 for x in UASratioRacesArray) / len(UASratioRacesArray)
    AwinsRace = [] # this list has the indeces of the Races in which nation A wins!
    AwinsRace.append([i for i in range(len(UASratioRacesArray)) if UASratioRacesArray[i] > 1])
    
    percentageBwins = np.sum(x < 1 for x in UASratioRacesArray) / len(UASratioRacesArray)
    BwinsRace = [] # this list has the indeces of the Races in which nation B wins!
    BwinsRace.append([i for i in range(len(UASratioRacesArray)) if UASratioRacesArray[i] < 1])

    return percentageAwins, percentageBwins, AwinsRace, BwinsRace

In [ ]:
# get percentage of races that A or B wins depending on type of race
percentageAwinsGeneral, percentageBwinsGeneral, AwinsGeneralRace, BwinsGeneralRace = racetypeToCheck(UASratioRaces)
percentageAwinsBilateral, percentageBwinsBilateral, AwinsBilateralRace, BwinsBilateralRace = racetypeToCheck(UASratiobilateral)
percentageAwinsARace, percentageBwinsARace, AwinsARace, BwinsARace = racetypeToCheck(UASratioARaces)
percentageAwinsBRace, percentageBwinsBRace, AwinsBRace, BwinsBRace = racetypeToCheck(UASratioBRaces)
percentageAwinsNoRace, percentageBwinsNoRace, AwinsNoRace, BwinsNoRace = racetypeToCheck(UASratioNoRaces)

In [ ]:
# plot some scenarios
df1 = pd.Series( outcomes['effectiveness UAS stock A'][onlyARaces[AwinsARace[0][4]]])
df2 = pd.Series( outcomes['effectiveness UAS stock B'][onlyARaces[AwinsARace[0][4]]])
df1.plot(label='Effectiveness systems A')
df2.plot(label='Effectiveness systems B')
plt.legend()
plt.title("Evolution of the effectiveness of autonomous systems")
plt.xlabel("Time Steps")
plt.ylabel("Effectiveness of systems (%)")
plt.show()

In [ ]:
# get indeces of races per different policy
noPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'no policy'].index
qualityPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'quality policy'].index
fundingPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'funding policy'].index
combinedPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'combined policy'].index

In [ ]:
# compute difference in parameter values for different policies

df2 = pd.DataFrame(experiments.iloc[noPolicyRaces].mean(),columns = ['No Policy Races'])

df2['Quality Policy Races'] = experiments.iloc[qualityPolicyRaces].mean()
df2['Funding Policy Races'] = experiments.iloc[fundingPolicyRaces].mean()
df2['Combined Policy Races'] = experiments.iloc[combinedPolicyRaces].mean()

df2['Relative Diff None vs Quality Policy %'] = 100* (df2['Quality Policy Races']  - df2['No Policy Races']) / df2['Quality Policy Races']
df2['Relative Diff None vs Funding Policy %'] = 100* (df2['Funding Policy Races']  - df2['No Policy Races']) / df2['Funding Policy Races']
df2['Relative Diff None vs Combined Policy %'] = 100* (df2['Combined Policy Races']  - df2['No Policy Races'])/df2['Combined Policy Races']


pd.set_option('display.max_rows', None)

# show those where the difference is more than 10%
pd.DataFrame(df2[abs(df2['Relative Diff None vs Funding Policy %']) > abs(1)]['Relative Diff None vs Funding Policy %'])

# Cluster Defense Spending

In [ ]:
nb_clusters = 3

distancesB = clusterer.calculate_cid(outcomes['New percentage of GDP B'][Races])
clusteringB = AgglomerativeClustering(n_clusters = nb_clusters).fit_predict(distancesB)

distancesA = clusterer.calculate_cid(outcomes['New percentage of GDP A'][Races])
clusteringA = AgglomerativeClustering(n_clusters = nb_clusters).fit_predict(distancesA)

In [ ]:
experiments['Cluster B'] = 4 # cluster 4 being the cluster for those scenarios that are not races
experiments['Cluster B'].iloc[Races] = clusteringB # add to dataframe
experiments['Cluster B'] = experiments['Cluster B'].astype('category') # change data type to categorical 
experiments['Cluster A'] = 4 # not being a race at all
experiments['Cluster A'].iloc[Races] = clusteringB
experiments['Cluster A'] = experiments['Cluster A'].astype('category')

In [ ]:
np.sum(np.where(experiments['Cluster A'] == 1, 1, 0) * Arace) #type of race in cluster

In [ ]:
np.sum(np.where(experiments['Cluster A'] == 1, 1, 0) * Brace)

In [ ]:
np.sum(np.where(experiments['Cluster A'] == 1, 1, 0) * bilateralrace)

In [ ]:
# plot all outcomes depending on clusters for defense spending B
race = {}
race['spending A'] = outcomes['New percentage of GDP A'][Races]
race['spending B']= outcomes['New percentage of GDP B'][Races]

figure = lines(experiments.iloc[Races], race, group_by = 'Cluster B')
plt.rcParams["figure.figsize"] =(10,10)
plt.show()

In [ ]:
# compute difference in parameter values of cluster membership 
df2 = pd.DataFrame(experiments.iloc[Races].mean(),columns = ['Races'])

df2['Normal'] = experiments.iloc[ScenariosNotRaces].mean()
df2['A: Cluster 0'] = experiments.iloc[ExpsNoExtrRaces.loc[ExpsNoExtrRaces['Cluster A'] == 0].index].mean()
df2['A: Cluster 1'] = experiments.iloc[ExpsNoExtrRaces.loc[ExpsNoExtrRaces['Cluster A'] == 1].index].mean()
df2['A: Cluster 2'] = experiments.iloc[ExpsNoExtrRaces.loc[ExpsNoExtrRaces['Cluster A'] == 2].index].mean()

df2['B: Cluster 0'] = experiments.iloc[ExpsNoExtrRaces.loc[ExpsNoExtrRaces['Cluster B'] == 0].index].mean()
df2['B: Cluster 1'] = experiments.iloc[ExpsNoExtrRaces.loc[ExpsNoExtrRaces['Cluster B'] == 1].index].mean()
df2['B: Cluster 2'] = experiments.iloc[ExpsNoExtrRaces.loc[ExpsNoExtrRaces['Cluster B'] == 2].index].mean()

df2['Relative Diff A 0 %'] = 100* (df2['A: Cluster 0']  - df2['Normal']) / df2['A: Cluster 0']
df2['Relative Diff A 1 %'] = 100* (df2['A: Cluster 1']  - df2['Normal']) / df2['A: Cluster 1']
df2['Relative Diff A 2 %'] = 100* (df2['A: Cluster 2']  - df2['Normal']) / df2['A: Cluster 2']

df2['Relative Diff B 0 %'] = 100* (df2['B: Cluster 0']  - df2['Normal']) / df2['B: Cluster 0']
df2['Relative Diff B 1 %'] = 100* (df2['B: Cluster 1']  - df2['Normal']) / df2['B: Cluster 1']
df2['Relative Diff B 2 %'] = 100* (df2['B: Cluster 2']  - df2['Normal']) / df2['B: Cluster 2']

pd.set_option('display.max_rows', None)

df2

# PRIM

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

In [ ]:
# prepare PRIM for regulation policies

noPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'no policy'].index
staticPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'static policy'].index
adaptivePolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'adaptive policy'].index
midwayPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'halfway policy'].index

staticPolicyRaces = [x - 30000 for x in staticPolicyRaces]
adaptivePolicyRaces  = [x - 10000 for x in adaptivePolicyRaces]
midwayPolicyRaces = [x - 20000 for x in midwayPolicyRaces]

# no policy
nprace = np.full((10000), False, dtype=bool)
nprace[noPolicyRaces] = True
# static regulation
sprace = np.full((10000), False, dtype=bool)
sprace[staticPolicyRaces] = True
# adapting regulation
aprace = np.full((10000), False, dtype=bool)
aprace[adaptivePolicyRaces] = True
#midway regulation
mprace = np.full((10000), False, dtype=bool)
mprace[midwayPolicyRaces] = True


In [ ]:
# prepare PRIM for spoiling strategies

noPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'no policy'].index
qualityPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'quality policy'].index
fundingPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'funding policy'].index
combinedPolicyRaces = experiments.iloc[Races].loc[experiments['policy'] == 'combined policy'].index

combinedPolicyRaces = [x - 30000 for x in combinedPolicyRaces]
qualityPolicyRaces  = [x - 10000 for x in qualityPolicyRaces]
fundingPolicyRaces = [x - 20000 for x in fundingPolicyRaces]

#no policy
nprace = np.full((10000), False, dtype=bool)
nprace[noPolicyRaces] = True
#combined strategy
cprace = np.full((10000), False, dtype=bool) 
cprace[combinedPolicyRaces] = True
#funding strategy
fprace = np.full((10000), False, dtype=bool) 
fprace[fundingPolicyRaces] = True
#quality strategy
qprace = np.full((10000), False, dtype=bool)
qprace[qualityPolicyRaces] = True

In [ ]:
# run prim, either with qrace, nprace, cprace, fprace, qprace, sprace, aprace or mprace and the corresponding experiments in dataframe
prim_alg = prim.Prim(experiments.loc[experiments['policy'] == 'quality policy'], qprace, threshold=0.02)

#find box
NPbox = prim_alg.find_box()

boi = 50 # choose box of interest
NPbox.show_tradeoff()
NPbox.inspect(boi, style='graph')

plt.show()

In [ ]:
# PRIM based on the fact whether arms race or not!

#normal races
race = np.full((nbscenarios), False, dtype=bool)
race[AntiRaces] = True
#bilateral races
bilateralrace = np.full((nbscenarios), False, dtype=bool)
bilateralrace[bilateralRaces] = True
#A driven Race
Arace = np.full((nbscenarios), False, dtype=bool)
Arace[onlyARaces] = True
#B driven race
Brace = np.full((nbscenarios), False, dtype=bool)
Brace[onlyBRaces] = True

prim_alg = prim.Prim(experiments, race, threshold=0.02) # race, bilateralrace, Arace or Brace

In [ ]:
#find box and investigate
box1 = prim_alg.find_box()

boi=20 # choose one box of interest
box1.show_tradeoff()
box1.inspect(boi, style='graph')

plt.show()